# 載入所需套件

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import re

#隱藏warning
requests.packages.urllib3.disable_warnings()

# 定義去除unicode字串函數

In [2]:
def process_content(input_str):
    return ''.join([c if ord(c)<=55291 else '_' for c in input_str])

# 定義轉換民國年到西元年字串函數

In [3]:
def process_time(input_str):
    return '/'.join([str(int(e)+1911) if i==0 else e for i,e in enumerate(input_str.split('.'))])

# 定義區分法院類別字串函數

In [4]:
def process_court(input_str):
    court_name=input_str.split(' ')[0]
    special_courts=['公務員懲戒委員會','智慧財產法院']
    for e in special_courts:
        if e in court_name:
            return e
    if '福建' in court_name:
        return court_name[2:6]
    else:
        return court_name[-4:]

# 定義區分地點字串函數

In [5]:
def process_location(input_str):
    if '福建' in input_str:
        return '金門'
    elif '地方' in input_str:
        return input_str.split(' ')[0][2:4]
    elif '高等' in input_str:
        if '分院' in input_str:
            return input_str.split(' ')[1][:2]
        elif '行政' in input_str:
            return input_str[:2]
        #臺灣高等法院(後面沒有接分院)
        else:
            return '臺北'
    #公務員懲戒委員會、智慧財產法院
    else:
        return '臺北'

# 定義司法院裁判書關鍵字查詢函數

In [6]:
def law_crawler(name):
    url='https://law.judicial.gov.tw/FJUD/default.aspx'

    #先get()取出會變動的參數
    session=requests.session()
    response=session.get(url,verify=False)
    soup=BeautifulSoup(response.text,'html.parser')

    VIEWSTATE=soup.select_one('#__VIEWSTATE')['value']
    VIEWSTATEGENERATOR=soup.select_one('#__VIEWSTATEGENERATOR')['value']
    VIEWSTATEENCRYPTED=soup.select_one('#__VIEWSTATEENCRYPTED')['value']
    EVENTVALIDATION=soup.select_one('#__EVENTVALIDATION')['value']

    form_data={'__VIEWSTATE':VIEWSTATE,
               '__VIEWSTATEGENERATOR':VIEWSTATEGENERATOR,
               '__VIEWSTATEENCRYPTED':VIEWSTATEENCRYPTED,
               '__EVENTVALIDATION':EVENTVALIDATION,
               'txtKW':name,
               'judtype':'JUDBOOK',
               'whosub':'0',
               'ctl00$cp_content$btnSimpleQry':'送出查詢'}

    #再post餵入form_data
    response=session.post(url,data=form_data,verify=False)
    soup=BeautifulSoup(response.text,'html.parser')
    #爬取q_value值
    q_value=soup.select_one('#hidQID')['value']

    #調整網址q_value值進入判決整理頁面
    url='https://law.judicial.gov.tw/FJUD/qryresultlst.aspx?q={}'.format(q_value)
    response=requests.get(url,verify=False)
    soup=BeautifulSoup(response.text,'html.parser')

    #紀錄總判決頁數，處理頁數可能只有一頁的情況
    try:
        pages=int(re.findall(r'/\s(\d+)\s',soup.select('div.pull-right')[0].text)[0])
    except:
        pages=1
    
    #最多爬500筆資料(25頁)
    if pages>25:
        pages=25
    
    dfs=[]
    #抓取每頁判決表格
    for page in tqdm(range(1,pages+1)):
        link=url+'&sort=DS&page={}&ot=in'.format(page)
        response=requests.get(link,verify=False)
        try:
            df=pd.read_html(response.text)[0]
            Names=pd.Series([name]*len(df[df.index%2!=0]))
            Contents=df[df.index%2!=0]['裁判日期'].apply(lambda x:process_content(x)).reset_index(drop=True)
            Times=df[df.index%2==0]['裁判日期'].apply(lambda x:process_time(x)).reset_index(drop=True)
            Titles=df[df.index%2==0]['裁判字號 （內容大小）'].reset_index(drop=True)
            CaseNos=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:int(re.findall(r'第 (\d+) 號',x)[0])).reset_index(drop=True)
            Sizes=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:re.findall(r'（(\w+)）',x)[0]).reset_index(drop=True)
            Types=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:re.findall(r'號(\w{2})',x)[0]).reset_index(drop=True)
            Judges=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:re.findall(r'號\w{2}(\w{2})（',x)[0]).reset_index(drop=True)
            Words=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:re.findall(r'度\s(\w+)\s字',x)[0]).reset_index(drop=True)
            Courts=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:process_court(x)).reset_index(drop=True)
            Locations=df[df.index%2==0]['裁判字號 （內容大小）'].apply(lambda x:process_location(x)).reset_index(drop=True)
            Causes=df[df.index%2==0]['裁判案由'].reset_index(drop=True)
            
            #建立小表格
            df=pd.DataFrame({'name':Names,
                             'time':Times,
                             'location':Locations,
                             'court':Courts,
                             'caseNo':CaseNos,
                             'type':Types,
                             'judge':Judges,
                             'word':Words,
                             'cause':Causes,
                             'size':Sizes,
                             'title':Titles,
                             'content':Contents})
            dfs.append(df)
            time.sleep(0.5)
        except:
            None
    #合併成大表格
    df=pd.concat(dfs,ignore_index=True)
    #改成字典格式
    data_list=df.to_dict(orient='records')
        
    return df,data_list

# 抓取指定司法院裁判書關鍵字查詢結果

In [7]:
df,data_list=law_crawler('蔡英文')

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:23<00:00,  1.06s/it]


In [8]:
df.head()

,name,time,location,court,caseNo,type,judge,word,cause,size,title,content
0,蔡英文,2020/09/09,臺南,地方法院,2841,刑事,判決,簡,賭博,5K,臺灣臺南地方法院 109 年度 簡 字第 2841 號刑事判決（5K）,...期間_接受不特定賭客以傳送訊息之方式下注_並以總統候選人蔡英文、韓國瑜2人選舉開票結果...
1,蔡英文,2020/09/08,橋頭,地方法院,473,刑事,判決,審易,竊盜,9K,臺灣橋頭地方法院 109 年度 審易 字第 473 號刑事判決（9K）,...離開龍翔大飯店等情_為被告所坦承_核與告訴人於警詢中證稱_「…因為我是蔡英文跟劉世芳的...
2,蔡英文,2020/09/07,高雄,地方法院,1,民事,判決,國簡上,國家賠償,18K,臺灣高雄地方法院 109 年度 國簡上 字第 1 號民事判決（18K）,...濟部法定代理人沈榮津交通部法定代理人林佳龍行政院原住民委員會法定代理人夷將‧拔路兒總統...
3,蔡英文,2020/08/31,高雄,高等法院,22,刑事,判決,附民上,損害賠償,2K,臺灣高等法院 高雄分院 109 年度 附民上 字第 22 號刑事判決（2K）,...按刑事訴訟諭知無罪、免訴或不受理之判決者_應以判決駁回原告之訴_刑事訴訟法第503 條...
4,蔡英文,2020/08/31,臺北,高等法院,148,民事,裁定,聲再,當選無效聲請再審,2K,臺灣高等法院 109 年度 聲再 字第 148 號民事裁定（2K）,...得提起再審之訴_自包括不得對確定裁定為再審之聲請_始符合該條之立法目的及選舉、罷免訴訟...


In [9]:
data_list

[{'name': '蔡英文',
  'time': '2020/09/09',
  'location': '臺南',
  'court': '地方法院',
  'caseNo': 2841,
  'type': '刑事',
  'judge': '判決',
  'word': '簡',
  'cause': '賭博',
  'size': '5K',
  'title': '臺灣臺南地方法院 109 年度 簡 字第 2841 號刑事判決（5K）',
  'content': '...期間_接受不特定賭客以傳送訊息之方式下注_並以總統候選人蔡英文、韓國瑜2人選舉開票結果為依據_賭法為如開票結果民主進步黨總統候選人蔡英文勝選達110萬票或100萬票_則洪郡澤應支付賭客下注金額85%或80%之彩金_如否_下注賭金即洪郡澤達所有。嗣於同日21時5分許_...'},
 {'name': '蔡英文',
  'time': '2020/09/08',
  'location': '橋頭',
  'court': '地方法院',
  'caseNo': 473,
  'type': '刑事',
  'judge': '判決',
  'word': '審易',
  'cause': '竊盜',
  'size': '9K',
  'title': '臺灣橋頭地方法院 109 年度 審易 字第 473 號刑事判決（9K）',
  'content': '...離開龍翔大飯店等情_為被告所坦承_核與告訴人於警詢中證稱_「…因為我是蔡英文跟劉世芳的特助_她有寫一封要給蔡英文的信委託我幫忙交給蔡英文。她和我一起回我所投宿之龍翔飯店。到龍翔飯店房間後就開始寫信_之後我便下樓去拿飲料上來請她喝_出門到回來大概花不到10分鐘_剩下時...'},
 {'name': '蔡英文',
  'time': '2020/09/07',
  'location': '高雄',
  'court': '地方法院',
  'caseNo': 1,
  'type': '民事',
  'judge': '判決',
  'word': '國簡上',
  'cause': '國家賠償',
  'size': '18K',
  'title': '臺灣高雄地方法院 109 年度 國簡上 字第 1 號民事判決（18K）